In [1]:
import math
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import lxml
import re
from random import seed
from collections import Counter
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import array
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import desc

C:\Users\MoJo\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
LinkOutdoor4x400m = "http://www.ustfccca.org/records-lists/outdoor-collegiate-all-time-bests?gender=2&sport=2&div=20527&top=25&event=19"

In [3]:
page = requests.get(LinkOutdoor4x400m)
soup = BeautifulSoup(page.text,'html')
soup

table = soup.find("table")

In [4]:
Header = soup.findAll('th')
Headers = []
for h in Header:
    Headers.append(h.text)

In [5]:
Headers

['Team', 'Institution', 'Mark', 'Date', '']

In [6]:
row_soup = soup.findAll('tr')

In [7]:
row_soup

[<tr><th style="width: 35%;">Team</th><th colspan="2" style="width: 30%;">Institution</th><th style="width: 25%;">Mark</th><th style="width: 10%;">Date</th><th></th></tr>,
 <tr><td colspan="100" style="font-weight: bolder;">UNOFFICIAL Top 25 Performers and Performances</td></tr>,
 <tr><td colspan="5"><div class="container-table" style="width: 100%;"><table class="embed" width="100%"><thead><tr></tr></thead><tr><td style="width: 35%;"><span>(1) Kentucky</span></td><td style="width: 5%;"><span class="group-logo-inline"><img alt="Kentucky" class="group-logo-inline" src="/images/logos/rankings-article/kentucky.png" title="Kentucky"/></span></td><td style="width: 25%;"> </td><td style="width: 25%;"><a href="https://www.tfrrs.org/results/74844/4559067" target="_blank">3:21.93</a><a href="https://www.youtube.com/watch?v=cQG3EN8aKEY" target="_blank">📺</a> </td><td style="width: 10%;text-align:right;" title="SEC Outdoor Championships (Oxford, Miss.)"><span class="mark-date" style="width: 10%;te

In [8]:
rows = []
for row in row_soup:
    row_data = []
    for cell in row.findAll('td'):
        row_data.append(cell.text)
    rows.append(row_data)

In [9]:
rows

[[],
 ['UNOFFICIAL Top 25 Performers and Performances'],
 ['(1) Kentucky\xa03:21.93📺 5/14/2022\xa0\xa0\xa0\xa0\xa0Karimah DAVIS, Dajour MILES, Abby STEINER, Alexis HOLMES',
  '(1) Kentucky',
  '',
  '\xa0',
  '3:21.93📺 ',
  '5/14/2022',
  '\xa0\xa0\xa0\xa0\xa0Karimah DAVIS, Dajour MILES, Abby STEINER, Alexis HOLMES'],
 [],
 ['(1) Kentucky', '', '\xa0', '3:21.93📺 ', '5/14/2022'],
 ['\xa0\xa0\xa0\xa0\xa0Karimah DAVIS, Dajour MILES, Abby STEINER, Alexis HOLMES'],
 ['(2) Texas A&M\xa03:22.01 5/14/2022\xa0\xa0\xa0\xa0\xa0Tierra ROBINSON-JONES, Jania MARTIN, Laila OWENS, Charokee YOUNG',
  '(2) Texas A&M',
  '',
  '\xa0',
  '3:22.01 ',
  '5/14/2022',
  '\xa0\xa0\xa0\xa0\xa0Tierra ROBINSON-JONES, Jania MARTIN, Laila OWENS, Charokee YOUNG'],
 [],
 ['(2) Texas A&M', '', '\xa0', '3:22.01 ', '5/14/2022'],
 ['\xa0\xa0\xa0\xa0\xa0Tierra ROBINSON-JONES, Jania MARTIN, Laila OWENS, Charokee YOUNG'],
 ['\xa0\xa0\xa0\xa0— TEXAS A&M {2}\xa03:22.34 6/12/2021\xa0\xa0\xa0\xa0\xa0Tierra ROBINSON-JONES (SO), 

In [10]:
len(rows)

1124

In [11]:
rows[1112]

['(25) Illinois', '', '\xa0', '3:29.28 ', '6/8/2013']

In [12]:
len(rows)

1124

In [13]:
RealRows = []

In [14]:
for i in range(0,1113):
    if len(rows[i]) > 1:
        RealRows.append(rows[i])
        

In [15]:
RealRows

[['(1) Kentucky\xa03:21.93📺 5/14/2022\xa0\xa0\xa0\xa0\xa0Karimah DAVIS, Dajour MILES, Abby STEINER, Alexis HOLMES',
  '(1) Kentucky',
  '',
  '\xa0',
  '3:21.93📺 ',
  '5/14/2022',
  '\xa0\xa0\xa0\xa0\xa0Karimah DAVIS, Dajour MILES, Abby STEINER, Alexis HOLMES'],
 ['(1) Kentucky', '', '\xa0', '3:21.93📺 ', '5/14/2022'],
 ['(2) Texas A&M\xa03:22.01 5/14/2022\xa0\xa0\xa0\xa0\xa0Tierra ROBINSON-JONES, Jania MARTIN, Laila OWENS, Charokee YOUNG',
  '(2) Texas A&M',
  '',
  '\xa0',
  '3:22.01 ',
  '5/14/2022',
  '\xa0\xa0\xa0\xa0\xa0Tierra ROBINSON-JONES, Jania MARTIN, Laila OWENS, Charokee YOUNG'],
 ['(2) Texas A&M', '', '\xa0', '3:22.01 ', '5/14/2022'],
 ['\xa0\xa0\xa0\xa0— TEXAS A&M {2}\xa03:22.34 6/12/2021\xa0\xa0\xa0\xa0\xa0Tierra ROBINSON-JONES (SO), Charokee YOUNG (FR), Jaevin REED (SR) 51.86, Athing MU (FR) 48.85',
  '\xa0\xa0\xa0\xa0— TEXAS A&M {2}',
  '',
  '\xa0',
  '3:22.34 ',
  '6/12/2021',
  '\xa0\xa0\xa0\xa0\xa0Tierra ROBINSON-JONES (SO), Charokee YOUNG (FR), Jaevin REED (SR) 51

In [16]:
RealRealRows = RealRows[::2]
RealRealRows

[['(1) Kentucky\xa03:21.93📺 5/14/2022\xa0\xa0\xa0\xa0\xa0Karimah DAVIS, Dajour MILES, Abby STEINER, Alexis HOLMES',
  '(1) Kentucky',
  '',
  '\xa0',
  '3:21.93📺 ',
  '5/14/2022',
  '\xa0\xa0\xa0\xa0\xa0Karimah DAVIS, Dajour MILES, Abby STEINER, Alexis HOLMES'],
 ['(2) Texas A&M\xa03:22.01 5/14/2022\xa0\xa0\xa0\xa0\xa0Tierra ROBINSON-JONES, Jania MARTIN, Laila OWENS, Charokee YOUNG',
  '(2) Texas A&M',
  '',
  '\xa0',
  '3:22.01 ',
  '5/14/2022',
  '\xa0\xa0\xa0\xa0\xa0Tierra ROBINSON-JONES, Jania MARTIN, Laila OWENS, Charokee YOUNG'],
 ['\xa0\xa0\xa0\xa0— TEXAS A&M {2}\xa03:22.34 6/12/2021\xa0\xa0\xa0\xa0\xa0Tierra ROBINSON-JONES (SO), Charokee YOUNG (FR), Jaevin REED (SR) 51.86, Athing MU (FR) 48.85',
  '\xa0\xa0\xa0\xa0— TEXAS A&M {2}',
  '',
  '\xa0',
  '3:22.34 ',
  '6/12/2021',
  '\xa0\xa0\xa0\xa0\xa0Tierra ROBINSON-JONES (SO), Charokee YOUNG (FR), Jaevin REED (SR) 51.86, Athing MU (FR) 48.85'],
 ['\xa0\xa0\xa0\xa0— KENTUCKY {2}\xa03:22.55 6/11/2022\xa0\xa0\xa0\xa0\xa0Karimah DAV

In [17]:
RealRealRows[1][4]

'3:22.01 '

In [18]:
len(RealRealRows)

283

In [19]:
OfficialTimes = []

In [20]:
for i in range(0,len(RealRealRows)):
    OfficialTimes.append(RealRealRows[i][4])
OfficialTimes

['3:21.93📺 ',
 '3:22.01 ',
 '3:22.34 ',
 '3:22.55 ',
 '3:22.55 ',
 '3:22.94 ',
 '3:23.13 ',
 '3:23.27 ',
 '3:23.30 ',
 '3:23.35 ',
 '3:23.35 ',
 '3:23.69 ',
 '3:23.75  ',
 '3:24.05📺 ',
 '3:24.21 ',
 '3:24.45 ',
 '3:24.54 ',
 '3:24.54 ',
 '3:24.55 ',
 '3:24.59 ',
 '3:24.72 ',
 '3:25.01 ',
 '3:25.05 ',
 '3:25.16 qq = Performance occurred during a non-final round. ',
 '3:25.26  ',
 '3:25.45 ',
 '3:25.48 qq = Performance occurred during a non-final round. ',
 '3:25.57 ',
 '3:25.58 qq = Performance occurred during a non-final round. ',
 '3:25.59 qq = Performance occurred during a non-final round. ',
 '3:25.63 qq = Performance occurred during a non-final round. ',
 '3:25.63 ',
 '3:25.64 ',
 '3:25.67 qq = Performance occurred during a non-final round. ',
 '3:25.78 ',
 '3:25.78  ',
 '3:25.79 ',
 '3:25.79 ',
 '3:25.84 qq = Performance occurred during a non-final round. ',
 '3:25.89 ',
 '3:25.91 qq = Performance occurred during a non-final round. ',
 '3:25.92 ',
 '3:25.93 qq = Performance occurr

In [21]:
len(OfficialTimes)

283

In [22]:
RealRealRows[0][1]

'(1) Kentucky'

In [23]:
Names = []

for i in range(0,len(OfficialTimes)):
    Names.append(RealRealRows[i][1])
Names

['(1) Kentucky',
 '(2) Texas A&M',
 '\xa0\xa0\xa0\xa0— TEXAS A&M {2}',
 '\xa0\xa0\xa0\xa0— KENTUCKY {2}',
 '(3) Arkansas',
 '(4) Texas',
 '(5) Oregon',
 '\xa0\xa0\xa0\xa0— TEXAS {2}',
 '\xa0\xa0\xa0\xa0— TEXAS A&M {3}',
 '\xa0\xa0\xa0\xa0— TEXAS {3}',
 '(6) Southern California',
 '\xa0\xa0\xa0\xa0— ARKANSAS {2}',
 '\xa0\xa0\xa0\xa0— TEXAS {4}',
 '\xa0\xa0\xa0\xa0— ARKANSAS {3}',
 '\xa0\xa0\xa0\xa0— TEXAS {5}',
 '\xa0\xa0\xa0\xa0— ARKANSAS {4}',
 '\xa0\xa0\xa0\xa0— SOUTHERN CALIFORNIA {2}',
 '\xa0\xa0\xa0\xa0— OREGON {2}',
 '\xa0\xa0\xa0\xa0— TEXAS A&M {4}',
 '(7) LSU',
 '\xa0\xa0\xa0\xa0— OREGON {3}',
 '(8) UCLA',
 '\xa0\xa0\xa0\xa0— TEXAS {6}',
 '\xa0\xa0\xa0\xa0— ARKANSAS {5}',
 '\xa0\xa0\xa0\xa0— LSU {2}',
 '(9) South Carolina',
 '\xa0\xa0\xa0\xa0— ARKANSAS {6}',
 '\xa0\xa0\xa0\xa0— TEXAS A&M {5}',
 '\xa0\xa0\xa0\xa0— TEXAS {7}',
 '\xa0\xa0\xa0\xa0— TEXAS {8}',
 '\xa0\xa0\xa0\xa0— TEXAS A&M {6}',
 '\xa0\xa0\xa0\xa0— TEXAS A&M {7}',
 '\xa0\xa0\xa0\xa0— TEXAS {9}',
 '\xa0\xa0\xa0\xa0—

In [24]:
len(Names)

283

In [25]:
OfficialReGexNames = []

In [26]:
for i in range(0,len(Names)):
    if Names[i].startswith("\xa0\xa0"):
        OfficialReGexNames.append(Names[i][4:])
    else:
        OfficialReGexNames.append(Names[i])
        
OfficialReGexNames

['(1) Kentucky',
 '(2) Texas A&M',
 '— TEXAS A&M {2}',
 '— KENTUCKY {2}',
 '(3) Arkansas',
 '(4) Texas',
 '(5) Oregon',
 '— TEXAS {2}',
 '— TEXAS A&M {3}',
 '— TEXAS {3}',
 '(6) Southern California',
 '— ARKANSAS {2}',
 '— TEXAS {4}',
 '— ARKANSAS {3}',
 '— TEXAS {5}',
 '— ARKANSAS {4}',
 '— SOUTHERN CALIFORNIA {2}',
 '— OREGON {2}',
 '— TEXAS A&M {4}',
 '(7) LSU',
 '— OREGON {3}',
 '(8) UCLA',
 '— TEXAS {6}',
 '— ARKANSAS {5}',
 '— LSU {2}',
 '(9) South Carolina',
 '— ARKANSAS {6}',
 '— TEXAS A&M {5}',
 '— TEXAS {7}',
 '— TEXAS {8}',
 '— TEXAS A&M {6}',
 '— TEXAS A&M {7}',
 '— TEXAS {9}',
 '— KENTUCKY {3}',
 '— SOUTH CAROLINA {2}',
 '— LSU {3}',
 '— TEXAS A&M {8}',
 '— KENTUCKY {4}',
 '— TEXAS A&M {9}',
 '— ARKANSAS {7}',
 '— ARKANSAS {8}',
 '— TEXAS A&M {10}',
 '— TEXAS {10}',
 '— ARKANSAS {9}',
 '— TEXAS A&M {11}',
 '— KENTUCKY {5}',
 '— OREGON {4}',
 '— SOUTHERN CALIFORNIA {3}',
 '— TEXAS A&M {12}',
 '— TEXAS A&M {13}',
 '— SOUTHERN CALIFORNIA {4}',
 '(10) Florida',
 '— FLORIDA {2}

In [27]:
CleanedNames = []

for i in range(0,len(RealRealRows)):
    if OfficialReGexNames[i].startswith("—"):
        CleanedNames.append(OfficialReGexNames[i][2:])
    if OfficialReGexNames[i].startswith("("):
        CleanedNames.append(OfficialReGexNames[i][4:])        
CleanedNames

['Kentucky',
 'Texas A&M',
 'TEXAS A&M {2}',
 'KENTUCKY {2}',
 'Arkansas',
 'Texas',
 'Oregon',
 'TEXAS {2}',
 'TEXAS A&M {3}',
 'TEXAS {3}',
 'Southern California',
 'ARKANSAS {2}',
 'TEXAS {4}',
 'ARKANSAS {3}',
 'TEXAS {5}',
 'ARKANSAS {4}',
 'SOUTHERN CALIFORNIA {2}',
 'OREGON {2}',
 'TEXAS A&M {4}',
 'LSU',
 'OREGON {3}',
 'UCLA',
 'TEXAS {6}',
 'ARKANSAS {5}',
 'LSU {2}',
 'South Carolina',
 'ARKANSAS {6}',
 'TEXAS A&M {5}',
 'TEXAS {7}',
 'TEXAS {8}',
 'TEXAS A&M {6}',
 'TEXAS A&M {7}',
 'TEXAS {9}',
 'KENTUCKY {3}',
 'SOUTH CAROLINA {2}',
 'LSU {3}',
 'TEXAS A&M {8}',
 'KENTUCKY {4}',
 'TEXAS A&M {9}',
 'ARKANSAS {7}',
 'ARKANSAS {8}',
 'TEXAS A&M {10}',
 'TEXAS {10}',
 'ARKANSAS {9}',
 'TEXAS A&M {11}',
 'KENTUCKY {5}',
 'OREGON {4}',
 'SOUTHERN CALIFORNIA {3}',
 'TEXAS A&M {12}',
 'TEXAS A&M {13}',
 'SOUTHERN CALIFORNIA {4}',
 ' Florida',
 'FLORIDA {2}',
 'TEXAS A&M {14}',
 'SOUTHERN CALIFORNIA {5}',
 'TEXAS {11}',
 'ARKANSAS {10}',
 ' Auburn',
 'SOUTH CAROLINA {3}',
 'TEXAS 

In [28]:
CleanedNames2 = []

for i in range(0,len(RealRealRows)):
    if CleanedNames[i].startswith(" "):
        CleanedNames2.append(CleanedNames[i][1:])
    else:
        CleanedNames2.append(CleanedNames[i])        
CleanedNames2

['Kentucky',
 'Texas A&M',
 'TEXAS A&M {2}',
 'KENTUCKY {2}',
 'Arkansas',
 'Texas',
 'Oregon',
 'TEXAS {2}',
 'TEXAS A&M {3}',
 'TEXAS {3}',
 'Southern California',
 'ARKANSAS {2}',
 'TEXAS {4}',
 'ARKANSAS {3}',
 'TEXAS {5}',
 'ARKANSAS {4}',
 'SOUTHERN CALIFORNIA {2}',
 'OREGON {2}',
 'TEXAS A&M {4}',
 'LSU',
 'OREGON {3}',
 'UCLA',
 'TEXAS {6}',
 'ARKANSAS {5}',
 'LSU {2}',
 'South Carolina',
 'ARKANSAS {6}',
 'TEXAS A&M {5}',
 'TEXAS {7}',
 'TEXAS {8}',
 'TEXAS A&M {6}',
 'TEXAS A&M {7}',
 'TEXAS {9}',
 'KENTUCKY {3}',
 'SOUTH CAROLINA {2}',
 'LSU {3}',
 'TEXAS A&M {8}',
 'KENTUCKY {4}',
 'TEXAS A&M {9}',
 'ARKANSAS {7}',
 'ARKANSAS {8}',
 'TEXAS A&M {10}',
 'TEXAS {10}',
 'ARKANSAS {9}',
 'TEXAS A&M {11}',
 'KENTUCKY {5}',
 'OREGON {4}',
 'SOUTHERN CALIFORNIA {3}',
 'TEXAS A&M {12}',
 'TEXAS A&M {13}',
 'SOUTHERN CALIFORNIA {4}',
 'Florida',
 'FLORIDA {2}',
 'TEXAS A&M {14}',
 'SOUTHERN CALIFORNIA {5}',
 'TEXAS {11}',
 'ARKANSAS {10}',
 'Auburn',
 'SOUTH CAROLINA {3}',
 'TEXAS A&

In [29]:
len(OfficialTimes)

283

In [30]:
len(OfficialReGexNames)

283

In [31]:
OfficialTimes[6][:7]

'3:23.13'

In [32]:
Times = []

for i in range(0,len(OfficialTimes)):
    Times.append(OfficialTimes[i][:7])
    
Times

['3:21.93',
 '3:22.01',
 '3:22.34',
 '3:22.55',
 '3:22.55',
 '3:22.94',
 '3:23.13',
 '3:23.27',
 '3:23.30',
 '3:23.35',
 '3:23.35',
 '3:23.69',
 '3:23.75',
 '3:24.05',
 '3:24.21',
 '3:24.45',
 '3:24.54',
 '3:24.54',
 '3:24.55',
 '3:24.59',
 '3:24.72',
 '3:25.01',
 '3:25.05',
 '3:25.16',
 '3:25.26',
 '3:25.45',
 '3:25.48',
 '3:25.57',
 '3:25.58',
 '3:25.59',
 '3:25.63',
 '3:25.63',
 '3:25.64',
 '3:25.67',
 '3:25.78',
 '3:25.78',
 '3:25.79',
 '3:25.79',
 '3:25.84',
 '3:25.89',
 '3:25.91',
 '3:25.92',
 '3:25.93',
 '3:25.95',
 '3:25.97',
 '3:25.99',
 '3:26.07',
 '3:26.09',
 '3:26.12',
 '3:26.17',
 '3:26.27',
 '3:26.28',
 '3:26.31',
 '3:26.31',
 '3:26.37',
 '3:26.38',
 '3:26.40',
 '3:26.46',
 '3:26.46',
 '3:26.49',
 '3:26.53',
 '3:26.54',
 '3:26.56',
 '3:26.57',
 '3:26.58',
 '3:26.60',
 '3:26.63',
 '3:26.64',
 '3:26.71',
 '3:26.72',
 '3:26.72',
 '3:26.73',
 '3:26.73',
 '3:26.73',
 '3:26.73',
 '3:26.74',
 '3:26.76',
 '3:26.85',
 '3:26.90',
 '3:26.90',
 '3:26.92',
 '3:26.92',
 '3:26.96',
 '3:

In [33]:
len(Times)

283

In [34]:
OutdoorWomens4x400mDF = pd.DataFrame({'Schools': CleanedNames2, 'Times': Times})

In [35]:
OutdoorWomens4x400mDF.to_csv('OutdoorWomens4x400m.csv', index=False)